In [ ]:
#get the tweets by author gender
#pip install gender_guesser

import gender_guesser.detector as gender
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

session = requests.Session()
session.auth = ('dhh19', 'heldig')
files = requests.get("http://vm1973.kaj.pouta.csc.fi/brexit/data/continuous_rehydrated/all_tweets_df.msg", auth=HTTPBasicAuth('dhh19', 'heldig'))
total_df = pd.read_msgpack(files.content)

d = gender.Detector()
total_df["gender"] = [d.get_gender(x.split()[0]) if x != "" else "unknown" for x in total_df["user_name"]]

In [ ]:
#get the tweets by MEPs

labour = tuple(requests.get("https://raw.githubusercontent.com/stcoats/DHH19_Hackathon/master/labour_mps.txt").text.split("\n"))
tories = tuple(requests.get("https://raw.githubusercontent.com/stcoats/DHH19_Hackathon/master/tory_mps.txt").text.split("\n"))
snp = tuple(requests.get("https://raw.githubusercontent.com/stcoats/DHH19_Hackathon/master/snp_mps.txt").text.split("\n"))
libdems = tuple(requests.get("https://raw.githubusercontent.com/stcoats/DHH19_Hackathon/master/libdem_mps.txt").text.split("\n"))
labour_df = total_df[total_df["user_screen_name"].isin((labour))]
tories_df = total_df[total_df["user_screen_name"].isin((tories))]
snp_df = total_df[total_df["user_screen_name"].isin((snp))]
libdems_df = total_df[total_df["user_screen_name"].isin((libdems))]

In [ ]:
#Frame of number of tweets in each language by user

l_df = total_df.groupby(["user_screen_name","lang"]).size().unstack().fillna(0).astype(int)

In [ ]:
#Tokenize the tweet texts, taking into account import possible emoji, japanese, or chinese text
#Words are converted to lower case except usernames, hashtags, urls (in case you want to plot them) 

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import tinysegmenter
segmenter = tinysegmenter.TinySegmenter()

from chinese_tokenizer.tokenizer import Tokenizer
jie_ba_tokenizer = Tokenizer().jie_ba_tokenizer
import pandas as pd

def tknz(row):
    line = row["text"]
    if row["lang"]=="ja":
        try:
            tokens = segmenter.tokenize(line)
            return tokens
        except:
            return 'error'
    elif row["lang"]=="zh":
        try:
            tokens = jie_ba_tokenizer(line)
            return tokens
        except:
            return 'error'
    else:
        try:
            tokens = [x.lower() if not x.startswith(("@","http","www","#","t.co")) else x for x in tokenizer.tokenize(line)]
            return tokens
        except:
            return 'error'

st_emojis= re.compile("[\U0001f476\U0001f9d2\U0001f466\U0001f467\U0001f9d1\U0001f468\U0001f469\U0001f9d3\U0001f474\U0001f475\U0001f46e\U0001f575\U0001f482\U0001f477\U0001f934\U0001f478\U0001f473\U0001f472\U0001f9d5\U0001f9d4\U0001f471\U0001f935\U0001f470\U0001f930\U0001f931\U0001f47c\U0001f385\U0001f936\U0001f9d9\U0001f9da\U0001f9db\U0001f9dc\U0001f9dd\U0001f64d\U0001f64e\U0001f645\U0001f646\U0001f481\U0001f64b\U0001f647\U0001f926\U0001f937\U0001f486\U0001f487\U0001f6b6\U0001f3c3\U0001f483\U0001f57a\U0001f9d6\U0001f9d7\U0001f9d8\U0001f6c0\U0001f6cc\U0001f574\U0001f3c7\U0001f3c2\U0001f3cc\U0001f3c4\U0001f6a3\U0001f3ca\U000026f9\U0001f3cb\U0001f6b4\U0001f6b5\U0001f938\U0001f93d\U0001f93e\U0001f939\U0001f933\U0001f4aa\U0001f448\U0001f449\U0000261d\U0001f446\U0001f595\U0001f447\U0000270c\U0001f91e\U0001f596\U0001f918\U0001f919\U0001f590\U0000270b\U0001f44c\U0001f44d\U0001f44e\U0000270a\U0001f44a\U0001f91b\U0001f91c\U0001f91a\U0001f44b\U0001f91f\U0000270d\U0001f44f\U0001f450\U0001f64c\U0001f932\U0001f64f\U0001f485\U0001f442\U0001f443]")

total_df["tokens"]=total_df.apply(tknz,axis=1)


In [ ]:
#make vectors of texts
tweet_w2v = Word2Vec(size=400, min_count=10,window=5, workers=8)
tweet_w2v.build_vocab([x for x in tqdm(total_df["tokens"])])
tweet_w2v.train([x for x in tqdm(total_df["tokens"])], total_examples=tweet_w2v.corpus_count, epochs=tweet_w2v.iter)

total_df_f = total_df[total_df["gender"] == "female"]
total_df_m = total_df[total_df["gender"] == "male"]

# Vectors for females
tweet_w2vF = Word2Vec(size=400, min_count=10,window=5, workers=8)
tweet_w2vF.build_vocab([x for x in tqdm(total_df_f["tokens"])])
tweet_w2vF.train([x for x in tqdm(total_df_f("tokens"))], total_examples=tweet_w2vF.corpus_count, epochs=tweet_w2vF.iter)

# Vectors for males
tweet_w2vM = Word2Vec(size=400, min_count=10,window=5, workers=8)
tweet_w2vM.build_vocab([x for x in tqdm(total_df_m["tokens"])])
tweet_w2vM.train([x for x in tqdm(total_df_m("tokens"))], total_examples=tweet_w2vM.corpus_count, epochs=tweet_w2vM.iter)

In [ ]:
blame = ["blame","disappointment","disappointed","dismay","dismayed","betray","betrayal","frustration","embarrassment"] #or whatever words you want
n_items = tweet_w2v.wv.index2word[0:10000] # first 10k words

# Getting a list of word vectors
word_vectorsF = [tweet_w2vF[w] for w in n_items] + [tweet_w2vF[w] for w in blame if w in tweet_w2v.wv.vocab.keys()]

# Convert from 400 to 2 dimensions 
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, perplexity = 50)
tsne_w2vF = tsne_model.fit_transform(word_vectorsF)


In [ ]:
tsne_dfF = pd.DataFrame(tsne_w2vF, columns=['x', 'y'])
tsne_dfF['words'] = n_items + [x for x in blame if x in tweet_w2vF.wv.vocab.keys()] 
tsne_dfF['freq'] = [tweet_w2vF.wv.vocab[w].count for w in n_items if w in tweet_w2vF.wv.vocab.keys()] + [tweet_w2vF.wv.vocab[w].count for w in blame if w in tweet_w2vF.wv.vocab.keys()]# + [tweet_w2vF.wv.vocab[w].count for w in nord_prof if w in tweet_w2vF.wv.vocab.keys()]
tsne_dfF['color'] = ["blue"]*10000 + ["red"] * len([x for x in blame if x in tweet_w2vF.wv.vocab.keys()]) 

# Plotting 
plot_tfidfF = bp.figure(plot_width=700, plot_height=600, title="Map of 10000 word vectors and blame words for females",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

plot_tfidfF.scatter(x='x', y='y', color = 'color', source=tsne_dfF)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words","freq": "@freq"}
show(plot_tfidf)

#Do the same for the male words or for other social groups

In [ ]:
# This fragment iterates through the compressed .jsonl files in our "continuous_rehydrated" folder and makes a big data frame. 
# Much of it is borrowed from Twarc
import sys
from dateutil.parser import parse as date_parse
from six import string_types

def get_headings():
    return [
      'id',
      'tweet_url',
      'created_at',
      'parsed_created_at',
      'user_screen_name',
      'text',
      'tweet_type',
      'coordinates',
      'hashtags',
      'retweeted_hashtags',
      'media',
      'urls',
      'favorite_count',
      'in_reply_to_screen_name',
      'in_reply_to_status_id',
      'in_reply_to_user_id',
      'lang',
      'place',
      'possibly_sensitive',
      'retweet_count',
      'retweet_or_quote_id',
      'retweet_or_quote_screen_name',
      'retweet_or_quote_user_id',
      'source',
      'user_id',
      'user_created_at',
      'user_default_profile_image',
      'user_description',
      'user_favourites_count',
      'user_followers_count',
      'user_friends_count',
      'user_listed_count',
      'user_location',
      'user_name',
      'user_statuses_count',
      'user_time_zone',
      'user_urls',
      'user_verified',
    ]


def get_row(t, excel = False):
    get = t.get
    user = t.get('user').get
    return [
      get('id_str'),
      tweet_url(t),
      get('created_at'),
      date_parse(get('created_at')),
      user('screen_name'),
      text(t) if not excel else clean_str(text(t)),
      rt_text(t) if not excel else clean_str(rt_text(t)),
      tweet_type(t),
      coordinates(t),
      hashtags(t),
      retweeted_hashtags(t),
      media(t),
      urls(t),
      get('favorite_count'),
      get('in_reply_to_screen_name'),
      get('in_reply_to_status_id'),
      get('in_reply_to_user_id'),
      get('lang'),
      place(t),
      get('possibly_sensitive'),
      get('retweet_count'),
      retweet_id(t),
      retweet_screen_name(t),
      retweet_user_id(t),
      get('source'),
      user('id_str'),
      user('created_at'),
      user('default_profile_image'),
      user('description') if not excel else clean_str(user('description')),
      user('favourites_count'),
      user('followers_count'),
      user('friends_count'),
      user('listed_count'),
      user('location') if not excel else clean_str(user('location')),
      user('name') if not excel else clean_str(user('name')),
      user('statuses_count'),
      user('time_zone'),
      user_urls(t),
      user('verified'),
    ]


def clean_str(string):
    if isinstance(string, string_types):
        return string.replace('\n', ' ').replace('\r', '')
    return None


def text(t):
    return t.get('full_text') or t.get('extended_tweet', {}).get('full_text') or t['text']

def rt_text(t):
    if "retweeted_status" in tweet:
        return tweet["retweeted_status"]["full_text"]

def coordinates(t):
    if 'coordinates' in t and t['coordinates']:
        return '%f %f' % tuple(t['coordinates']['coordinates'])
    return None


def hashtags(t):
    return ' '.join([h['text'] for h in t['entities']['hashtags']])

def retweeted_hashtags(t):
    if 'retweeted_status' in t and t['retweeted_status']:
        return ' '.join([h['text'] for h in t['retweeted_status']['entities']['hashtags']])

def media(t):
    if 'extended_entities' in t and 'media' in t['extended_entities']:
        return ' '.join([h['media_url_https'] for h in t['extended_entities']['media']])
    elif 'media' in t['entities']: 
        return ' '.join([h['media_url_https'] for h in t['entities']['media']])
    else:
        return None


def urls(t):
    return ' '.join([h['expanded_url'] or '' for h in t['entities']['urls']])


def place(t):
    if 'place' in t and t['place']:
        return t['place']['full_name']


def retweet_id(t):
    if 'retweeted_status' in t and t['retweeted_status']:
        return t['retweeted_status']['id_str']
    elif 'quoted_status' in t and t['quoted_status']:
        return t['quoted_status']['id_str']


def retweet_screen_name(t):
    if 'retweeted_status' in t and t['retweeted_status']:
        return t['retweeted_status']['user']['screen_name']
    elif 'quoted_status' in t and t['quoted_status']:
        return t['quoted_status']['user']['screen_name']


def retweet_user_id(t):
    if 'retweeted_status' in t and t['retweeted_status']:
        return t['retweeted_status']['user']['id_str']
    elif 'quoted_status' in t and t['quoted_status']:
        return t['quoted_status']['user']['id_str']


def tweet_url(t):
    return "https://twitter.com/%s/status/%s" % (t['user']['screen_name'], t['id_str'])


def user_urls(t):
    u = t.get('user')
    if not u:
        return None
    urls = []
    if 'entities' in u and 'url' in u['entities'] and 'urls' in u['entities']['url']:
        for url in u['entities']['url']['urls']:
            if url['expanded_url']:
                urls.append(url['expanded_url'])
    return ' '.join(urls)


def tweet_type(t):
    # Determine the type of a tweet
    if t.get('in_reply_to_status_id'):
        return 'reply'
    if 'retweeted_status' in t:
        return 'retweet'
    if 'quoted_status' in t:
        return 'quote'
    return 'original'

import json
import glob
files = glob.glob("/home/cloud-user/Hackathon/brexit/data/continuous_rehydrated/*.jsonl") 

td = []
for x in files:
    with open(x) as f:
        for line in f:
            tweet = json.loads(line)
            td.append(get_row(tweet))
            
total_df = pd.DataFrame(td,columns = [
      'id',
      'tweet_url',
      'created_at',
      'parsed_created_at',
      'user_screen_name',
      'text',
      'retweeted_text',
      'tweet_type',
      'coordinates',
      'hashtags',
      'retweeted_hashtags',
      'media',
      'urls',
      'favorite_count',
      'in_reply_to_screen_name',
      'in_reply_to_status_id',
      'in_reply_to_user_id',
      'lang',
      'place',
      'possibly_sensitive',
      'retweet_count',
      'retweet_or_quote_id',
      'retweet_or_quote_screen_name',
      'retweet_or_quote_user_id',
      'source',
      'user_id',
      'user_created_at',
      'user_default_profile_image',
      'user_description',
      'user_favourites_count',
      'user_followers_count',
      'user_friends_count',
      'user_listed_count',
      'user_location',
      'user_name',
      'user_statuses_count',
      'user_time_zone',
      'user_urls',
      'user_verified',
    ])